In [ ]:
# load a dataframe

import pandas as pd

df = pd.read_csv('dataset_pca_3_components_personality.csv', sep=';')
df.head()

,PC1,PC2,PC3,Personality
0,0.425156,0.024034,0.255252,0.0
1,-0.822265,-0.213964,0.008904,1.0
2,-0.545634,-0.053308,0.213832,1.0
3,0.908568,-0.042458,0.072104,0.0
4,0.404915,0.079302,0.065595,0.0


In [ ]:
# with the results for each iteration from 1 up to the total number of features. 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Separate features and target
X = df.drop("Result", axis=1)
y = df["Result"]

# Scale numerical features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize KNN model
knn = KNeighborsClassifier()

# Initialize Sequential Feature Selector with forward selection
sfs = SequentialFeatureSelector(knn, direction='forward', cv=5)

results_df = pd.DataFrame(columns=['Num_Features', 'Selected_Features', 'F1_Score'])

for i in range(1, X_train.shape[1] + 1):
    sfs.n_features_to_select = i
    sfs.fit(X_train, y_train)

    selected_features_mask = sfs.get_support()
    selected_features = X_train.columns[selected_features_mask]

    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    knn.fit(X_train_selected, y_train)
    y_pred = knn.predict(X_test_selected)

    f1 = f1_score(y_test, y_pred, average='macro')

    results_df = pd.concat([results_df, pd.DataFrame([{'Num_Features': i, 'Selected_Features': list(selected_features), 'F1_Score': f1}])], ignore_index=True)

results_df


/tmp/ipython-input-9-2011141456.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{'Num_Features': i, 'Selected_Features': list(selected_features), 'F1_Score': f1}])], ignore_index=True)


ValueError: n_features_to_select must be < n_features.

In [ ]:
results_df

,Iteration,Selected Features,F1 Score (Macro Avg)
0,1,[Troponin],0.857875
1,2,"[Troponin, Blood sugar]",0.821985
2,3,"[Troponin, Blood sugar, Gender]",0.785340
3,4,"[Troponin, Blood sugar, Gender, Age]",0.764372
4,5,"[Troponin, Blood sugar, Gender, Age, Diastolic...",0.764165
5,6,"[Troponin, Blood sugar, Gender, Age, Diastolic...",0.685441


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Separate features and target variable
X = df.drop("Personality", axis=1)
y = df["Personality"]

# Scale numerical features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize the Gradient Boosting model
gb = GradientBoostingClassifier()

# Initialize the feature selector with forward selection
sfs = SequentialFeatureSelector(gb, direction='forward', cv=5)

# Initialize DataFrame to store results
results_df = pd.DataFrame({
    'Num_Features': pd.Series(dtype='int'),
    'Selected_Features': pd.Series(dtype='object'),
    'F1_Score': pd.Series(dtype='float')
})


# Iterate over the number of features to select (up to n-1 to avoid error)
for i in range(1, X_train.shape[1]):
    sfs.n_features_to_select = i
    sfs.fit(X_train, y_train)

    selected_features_mask = sfs.get_support()
    selected_features = X_train.columns[selected_features_mask]

    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    gb.fit(X_train_selected, y_train)
    y_pred = gb.predict(X_test_selected)

    f1 = f1_score(y_test, y_pred, average='macro')

    results_df = pd.concat(
        [results_df, pd.DataFrame([{
            'Num_Features': i,
            'Selected_Features': list(selected_features),
            'F1_Score': f1
        }])],
        ignore_index=True
    )

# Display results
results_df


,Num_Features,Selected_Features,F1_Score
0,1,[PC1],0.899070
1,2,"[PC1, PC2]",0.901126
